In [1]:
import maidenhead
import datetime
import pandas as pd
import dask.dataframe as dadf
import glob
import pyarrow.parquet as pq
import pyarrow as pa
from tqdm import tqdm
import easier as ezr
import duckdb
import os
# import datashader.spatial.points as dsp

In [4]:
available_cols = [
    'spot_id',
    'timestamp',
    'reporter',
    'reporters_grid',
    'snr',
    'frequency',
    'call_sign',
    'grid',
    'power',
    'drift',
    'distance',
    'azimuth',
    'band',
    'version',
    'code'
]

df = pd.read_csv('./data/wsprnet/2020/wsprspots-2020-01_aa.csv.gz', names=available_cols,)
df.head()


,spot_id,timestamp,reporter,reporters_grid,snr,frequency,call_sign,grid,power,drift,distance,azimuth,band,version,code
0,1913207476,1577836800,DP0GVN,IB59uh,-23,7.040077,2E0ETU,IO81wv,27,0,13640,182,7,NaN,0
1,1913208199,1577836800,IW2NKE,JN63np,-10,7.040077,2E0ETU,IO81wv,27,0,1458,123,7,NaN,0
2,1913208025,1577836800,IZ3EAW/B,JN55xi,-20,7.040029,2E0ETU,IO81wv,27,0,1262,120,7,1.3 Kiwi,0
3,1913206488,1577836800,KF4TEK,FM18dr,-22,7.040079,2E0ETU,IO81wv,27,0,5810,287,7,2.1.2,0
4,1913208961,1577836800,OE9HLH,JN47um,-25,7.040077,2E0ETU,IO81wv,27,0,978,115,7,NaN,0


In [8]:
conn = duckdb.connect(database=':memory:', read_only=False)
conn.register('mydf', df)

In [16]:
conn.execute('create table rob as select * from mydf')


In [22]:
conn.execute('select count(spot_id) from rob')
xxx = conn.fetchdf()
xxx

,count(spot_id)
0,500000


In [12]:
class Duck:
    
    FILE_NAME = './data/wsprnet/spots.db'
    
    def __init__(self, overwrite=False):
        if overwrite and os.path.isfile(self.FILE_NAME):
            os.unlink(self.FILE_NAME)
        
    def connection(self):
        db_exists = os.path.isfile(self.FILE_NAME)
        conn = duckdb.connect(self.FILE_NAME)
        if not db_exists:
            self.create_table(conn)
        return conn
        
            
    def create_table(self, connection):
        cmd = """CREATE TABLE spots(
            spot_id INT8,
            timestamp INT8,
            reporter VARCHAR,
            reporters_grid VARCHAR,
            snr INT2,
            frequency DOUBLE,
            call_sign VARCHAR,
            grid VARCHAR,
            power INT2,
            drift INT2,
            distance INT8,
            azimuth INT8,
            band INT2,
            version VARCHAR,
            code INT2
        )"""
        connection.execute(cmd)
        
    def import_file(self, file_name):
        df = pd.read_csv(file_name)
        conn = self.connection()
        conn.register('df', df)
        cmd = f"""
            INSERT INTO spots SELECT * FROM df
        """
        conn.execute(cmd)
        conn.close()
        
    def create_index(self):
        cmd = """
            CREATE INDEX timestamp_ind ON spots (timestamp)
        """
        conn = self.connection()
        conn.execute(cmd)
        conn.close()
        
        
        
duck = Duck(overwrite=True)
files = glob.glob('./data/wsprnet/2020/wsprspots-2020-01_a*.csv.gz')
for file in tqdm(files):
    duck.import_file(file)
print('creating index')
with ezr.Timer('indexing'):
    duck.create_index()

    
        
            
        

100%|██████████| 26/26 [03:57<00:00,  9.15s/it]


creating index
__time__,16.058735,indexing


In [10]:
with ezr.Timer('indexing'):
    Duck().create_index()

__time__,0.070486,indexing


In [22]:
with ezr.Timer('connecting'):
    conn = duckdb.connect(Duck.FILE_NAME)
# conn.execute('select count(*) from spots')
# conn.execute('select * from spots ORDER BY timestamp DESC LIMIT 1000')

with ezr.Timer('querying'):
#     conn.execute('select * from spots WHERE timestamp = 1578123360')
    conn.execute("PRAGMA index_info('spots')")
    dfx = conn.fetchdf()
    conn.close()
dfx.tail()

__time__,0.099694,connecting


RuntimeError: Parser: Unrecognized PRAGMA keyword: index_info

In [ ]:
conn = duckdb.connect(database=':memory:', read_only=False)
conn.execute()
